In [22]:
import torch
import torch.nn as nn
print(torch.device('cuda:0'))
torch.cuda.get_device_name(0)
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
print(torch.__version__)

cuda:0
1.0.0


In [0]:
# Hyperparams
# 28 x 28 (squashed)
input_size = 784
# Number of nodes at hidden layer
hidden_size = 500
# Number of classes
num_classes = 10
# time steps
num_epochs = 20
# the size of input data from one iteration
batch_size = 100
# learning rate
lr = 1e-3

In [6]:
# mnist data
train_data = dsets.MNIST(root='./data', 
                         train=True,transform=transforms.ToTensor(), download=True)

test_data = dsets.MNIST(root='./data', train=False,
                       transform=transforms.ToTensor())

Processing...
Done!


In [0]:
# load data
train_gen = torch.utils.data.DataLoader(dataset=train_data,
                                       batch_size=batch_size,
                                       shuffle=True)

test_gen = torch.utils.data.DataLoader(dataset=test_data,
                                      batch_size=batch_size,
                                      shuffle=False)

In [0]:
# model
class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)
   
  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [0]:
# build
net = Net(input_size, hidden_size=hidden_size, num_classes=num_classes)
if torch.cuda.is_available():
  net.cuda()

In [0]:
# Adam Backprop and Loss Func
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [24]:
# train
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_gen):
    images = Variable(images.view(-1, 28*28)).cuda()
    labels = Variable(labels).cuda()
    
    optimizer.zero_grad()
    outputs = net(images)
    loss = loss_function(outputs,labels)
    loss.backward()
    optimizer.step()
    if (i + 1) % 100 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
           % (epoch+1, num_epochs, i+1, len(train_data)//batch_size, loss.item()))  # change loss.data[0] to loss.item() for PyTorch version 1.0

Epoch [1/20], Step [100/600], Loss: 0.1765
Epoch [1/20], Step [200/600], Loss: 0.2097
Epoch [1/20], Step [300/600], Loss: 0.1583
Epoch [1/20], Step [400/600], Loss: 0.1743
Epoch [1/20], Step [500/600], Loss: 0.0825
Epoch [1/20], Step [600/600], Loss: 0.1185
Epoch [2/20], Step [100/600], Loss: 0.1307
Epoch [2/20], Step [200/600], Loss: 0.0322
Epoch [2/20], Step [300/600], Loss: 0.1010
Epoch [2/20], Step [400/600], Loss: 0.1301
Epoch [2/20], Step [500/600], Loss: 0.0772
Epoch [2/20], Step [600/600], Loss: 0.1096
Epoch [3/20], Step [100/600], Loss: 0.0411
Epoch [3/20], Step [200/600], Loss: 0.0297
Epoch [3/20], Step [300/600], Loss: 0.0892
Epoch [3/20], Step [400/600], Loss: 0.0360
Epoch [3/20], Step [500/600], Loss: 0.0660
Epoch [3/20], Step [600/600], Loss: 0.2363
Epoch [4/20], Step [100/600], Loss: 0.0358
Epoch [4/20], Step [200/600], Loss: 0.0682
Epoch [4/20], Step [300/600], Loss: 0.0205
Epoch [4/20], Step [400/600], Loss: 0.0174
Epoch [4/20], Step [500/600], Loss: 0.0353
Epoch [4/20

In [25]:
# Eval Accuracy
correct = 0
total = 0
for images, labels in test_gen:
  images = Variable(images.view(-1, 28*28)).cuda()
  labels = labels.cuda()
  
  output = net(images)
  _, predicted = torch.max(output, 1)
  correct += (predicted == labels).sum()
  total += labels.size(0)

print("accuracy of model: %.3f %%" % ((100*correct)/(total+1)))

accuracy of model: 98.000 %
